# Packages and Imports

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install pydicom
!pip install nipype
!pip install dcm2niix
!pip install nibabel
!pip install pycox
!pip install torchtuples
!pip install lifelines


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 5.0 MB/s 
     |████████████████████████████████| 5.1 MB 29.5 MB/s 
     |████████████████████████████████| 139 kB 48.0 MB/s 
     |████████████████████████████████| 421 kB 50.7 MB/s 
     |████████████████████████████████| 500 kB 51.2 MB/s 
     |████████████████████████████████| 41 kB 525 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20220715-cp38-cp38-linux_x86_64.whl size=601060 sha256=fa2421be966

In [4]:
import pydicom as di 
import os
from os import listdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from skimage.transform import resize
from subprocess import call
import re
from functools import reduce
import shutil
import nibabel as nib
from PIL import Image
import torch
import torchvision.transforms as transforms


#Model 1 
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 
import torchtuples as tt
from pycox.datasets import metabric
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv


import scipy.io

#Model 3

import sys, os, datetime, h5py
import numpy as np
from scipy import io
import pandas as pd
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input,InputLayer, Dense,  Dropout, Activation, Concatenate, Lambda
#from tensorflow.python.keras.summary import merge
# from tensorflow.keras.utils import plot_model, multi_gpu_model
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from functools import partial, update_wrapper
from tensorflow.keras import optimizers

from sklearn.preprocessing import StandardScaler

from lifelines.utils import concordance_index


import nipype.interfaces.fsl as fsl


from nipype.interfaces import fsl
from nipype.testing import example_data

In [5]:
path="/content/content/ADNI"

In [19]:
mode='mat'
modelMode='eval'

# Metadata

In [6]:
metadata0 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year0/healthy_and_ad_patient_0_12_01_2022.csv', sep=',', header=0)
metadata1 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year1/healthy_and_ad_patient_1_12_01_2022.csv', sep=',', header=0)
metadata2 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year2/healthy_and_ad_patient_2_12_01_2022.csv', sep=',', header=0)
metadata3 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year3/healty_and_ad_patient_3_12_01_2022.csv', sep=',', header=0)
metadata4 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year4/healthy_and_ad_patient_4_12_01_2022.csv', sep=',', header=0)
metadata5 = pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Year5/heathy_and_ad_patient_5_12_01_2022.csv', sep=',', header=0)

In [7]:
T=6

In [ ]:
genetic_data=pd.read_csv('/content/content/ADNIGenetic.csv')

In [ ]:
metadata=pd.read_csv('/content/gdrive/MyDrive/dataset/Survival/Merged/healthy_and_ad_patient.csv', sep=',', header=0)

# Model 1


## Load data

In [ ]:
durations=metadata['durations']
event=metadata['event']
covariates=metadata['covariates']

In [15]:
#replace with your path
if mode == 'mat':
  x_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/basic_data/features_basic_train.mat')
  x_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/basic_data/features_basic_test.mat')
  y_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/basic_data/y_basic_train.mat')
  y_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/basic_data/y_basic_test.mat')
  x_train = np.array( x_train["data"], dtype = 'float32' )
  x_test = np.array( x_test["data"], dtype = 'float32' )
  y_train = np.array( y_train["data"], dtype = 'float32' )
  y_test = np.array( y_test["data"], dtype = 'float32' )

else:
  labels=list(zip(durations, event))
  x_train,x_test,y_train,y_test=train_test_split(covariates,labels,test_size=0.2)
d_train=np.array([y[0] for y in y_train], dtype='float32')
e_train=np.array([y[1] for y in y_train], dtype='float32')
d_test=np.array([y[0] for y in y_test], dtype='float32')
e_test=np.array([y[1] for y in y_test], dtype='float32')

## Network

In [11]:
class CNN(Module):
  def __init__(self, numChannels, classes):
    # call the parent constructor
    super(CNN, self).__init__()
    # initialize first set of CONV => RELU => POOL layers
    self.conv1 = Conv2d(in_channels=numChannels, out_channels=20,
      kernel_size=(7, 7))
    #224-5 + 1= 220
    self.relu1 = ReLU()
    self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    #220/2=110
    #110/2=55
    # initialize second set of CONV => RELU => POOL layers
    self.conv2 = Conv2d(in_channels=20, out_channels=50,
      kernel_size=(5, 5))
    #55-5+1=51
    self.relu2 = ReLU()
    self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
    #51/2=25
    self.conv3 = Conv2d(in_channels=50, out_channels=50,
      kernel_size=(5, 5))
    #25-5+1=21
    # initialize first (and only) set of FC => RELU layers
    self.fc1 = Linear(in_features=21*21*50, out_features=500)
    self.relu3 = ReLU()
    # initialize our softmax classifier
    self.fc2 = Linear(in_features=500, out_features=classes)
    self.logSoftmax = LogSoftmax(dim=1)

  def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.maxpool1(x)
    x = self.maxpool1(x)
    # pass the output from the previous layer through the second
    # set of CONV => RELU => POOL layers
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.maxpool2(x)
    x = self.conv3(x)
    # flatten the output from the previous layer and pass it
    # through our only set of FC => RELU layers
    x = flatten(x, 1)
    x = self.fc1(x)
    x = self.relu3(x)
    # pass the output to our softmax classifier to get our output
    # predictions
    x = self.fc2(x)
    output = self.logSoftmax(x)
    # return the output predictions
    return output


### Network Params

In [12]:
numClasses=T
numChannels=3

## Training

In [28]:
net=CNN(3, 6)
labtrans = DeepHitSingle.label_transform(T)
labels = labtrans.fit_transform(d_train,e_train)
model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)
model.load_net('/content/gdrive/MyDrive/dataset/Survival/Models/basic.pt')
batch_size = 12


In [17]:
lr_finder = model.lr_finder(x_train, labels, batch_size, tolerance=3)
lr_finder.get_best_lr()

0.0001321941148466034

In [25]:
if modelMode == 'train':
  model.optimizer.set_lr(0.00008)
  batch_size = 12
  epochs = 100
  callbacks = [tt.callbacks.EarlyStopping()]
  log = model.fit(x_train, labels, batch_size, epochs,callbacks)

0:	[13s / 13s],		train_loss: 0.1028
1:	[11s / 24s],		train_loss: 0.0874
2:	[11s / 35s],		train_loss: 0.0809
3:	[12s / 47s],		train_loss: 0.0668
4:	[11s / 58s],		train_loss: 0.0624
5:	[13s / 1m:11s],		train_loss: 0.0557
6:	[11s / 1m:22s],		train_loss: 0.0512
7:	[11s / 1m:34s],		train_loss: 0.0442
8:	[14s / 1m:48s],		train_loss: 0.0381
9:	[13s / 2m:1s],		train_loss: 0.0349


## Evaluation

In [29]:
surv = model.interpolate(6).predict_surv_df(x_test)
ev = EvalSurv(surv, d_test, e_test, censor_surv='km')
ev.concordance_td('antolini')

0.6956521739130435

## Save model and data

In [ ]:
model.save_net('basic.pt')
scipy.io.savemat('/content/basic_data/features_basic_test.mat', {"data":x_test})
scipy.io.savemat('/content/basic_data/features_basic_train.mat', {"data":x_train})
scipy.io.savemat('/content/basic_data/y_basic_train.mat', {"data":y_train})
scipy.io.savemat('/content/basic_data/y_basic_test.mat', {"data":y_test})

# Model 2

## Load Data

In [ ]:
durations=metadata['durations']
event=metadata['event']
covariates=metadata['covariates']

In [125]:
#replace with your path
if mode == 'mat':
  x_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/longitudnal_data/features_train.mat')
  x_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/longitudnal_data/features_test.mat')
  y_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/longitudnal_data/y_train.mat')
  y_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/longitudnal_data/y_test.mat')
  x_train = np.array( x_train["data"], dtype = 'float32' )
  x_test = np.array( x_test["data"], dtype = 'float32' )
  y_train = np.array( y_train["data"], dtype = 'float32' )
  y_test = np.array( y_test["data"], dtype = 'float32' )

else:
  labels=list(zip(durations, event))
  x_train,x_test,y_train,y_test=train_test_split(covariates,labels,test_size=0.2)
d_train=np.array([y[0] for y in y_train], dtype='float32')
e_train=np.array([y[1] for y in y_train], dtype='float32')
d_test=np.array([y[0] for y in y_test], dtype='float32')
e_test=np.array([y[1] for y in y_test], dtype='float32')


In [126]:
print(x_train.shape, x_test.shape, d_train.shape, d_test.shape, e_train.shape, e_test.shape)

(201, 125440) (51, 125440) (201,) (51,) (201,) (51,)


## Network

### Renet

In [30]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-2])
for param in feature_extractor.parameters():
    param.requires_grad = False

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

### Network params

In [40]:
in_features = 5*25088
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1

In [45]:
labtrans = DeepHitSingle.label_transform(6)
labels = labtrans.fit_transform(d_train,e_train)
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
model2 = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

In [47]:
model2.load_net('/content/gdrive/MyDrive/dataset/Survival/Models/longitudnal.pt')

MLPVanilla(
  (net): Sequential(
    (0): DenseVanillaBlock(
      (linear): Linear(in_features=125440, out_features=32, bias=True)
      (activation): ReLU()
      (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): DenseVanillaBlock(
      (linear): Linear(in_features=32, out_features=32, bias=True)
      (activation): ReLU()
      (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (2): Linear(in_features=32, out_features=6, bias=True)
  )
)

## Training

In [42]:
if mode != 'mat':
  model2.optimizer.set_lr(0.00007)
  epochs = 100
  callbacks = [tt.callbacks.EarlyStopping()]
  log = model2.fit(x_train, labels, 12, epochs, callbacks)

0:	[1s / 1s],		train_loss: 0.1550
1:	[1s / 2s],		train_loss: 0.1221
2:	[1s / 3s],		train_loss: 0.0979
3:	[1s / 4s],		train_loss: 0.0950
4:	[1s / 5s],		train_loss: 0.0883
5:	[1s / 6s],		train_loss: 0.0817
6:	[1s / 7s],		train_loss: 0.0773
7:	[1s / 8s],		train_loss: 0.0725
8:	[1s / 10s],		train_loss: 0.0690
9:	[1s / 11s],		train_loss: 0.0688


## Evaluation

In [48]:
surv = model2.interpolate(6).predict_surv_df(x_test)
ev = EvalSurv(surv, d_test, e_test, censor_surv='km')
ev.concordance_td('antolini')

0.691304347826087

## Save model and data

In [ ]:
scipy.io.savemat('/content/longitudnal_data/features_test.mat', {"data":x_test})
scipy.io.savemat('/content/longitudnal_data/features_train.mat', {"data":x_train})
scipy.io.savemat('/content/longitudnal_data/y_train.mat', {"data":y_train})
scipy.io.savemat('/content/longitudnal_data/y_test.mat', {"data":y_test})

model.save_net('longitudnal.pt')

# Model 3

## Load Data

In [ ]:
x=metadata['covariates']
E=metadata['event']
mask=metadata['mask']

In [75]:
#replace with your path
if mode == 'mat':
  x_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/features_train.mat')
  x_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/features_test.mat')
  y_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/y_train.mat')
  y_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/y_test.mat')
  mask_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/mask_test.mat')
  mask_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/mask_train.mat')
  e_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/e_train.mat')
  e_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/deep_surv_data/e_test.mat')


  x_train = np.array( x_train["data"], dtype = 'float32' )
  x_test = np.array( x_test["data"], dtype = 'float32' )
  y_train = np.array( y_train["data"], dtype = 'float32' )
  y_test = np.array( y_test["data"], dtype = 'float32' )
  mask_train = np.array( mask_train["data"], dtype = 'float32' )
  mask_test = np.array( mask_test["data"], dtype = 'float32' )
  e_train = np.array( e_train["data"], dtype = 'float32' )
  e_test = np.array( e_test["data"], dtype = 'float32' )


else:
  x_train,x_test,y_train,y_test,mask_train,mask_test,e_train,e_test=train_test_split(x,E,mask,event,test_size=0.2)


## Network

### Resnet

In [67]:
#Resnet
fe = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
fe=torch.nn.Sequential(*(list(fe.children())[:-1]))
fe.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

### Network Params

In [94]:
nn=254
n_features=512
DROPOUT_RATIO = 0.5
J = 5

In [93]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

with tf.device("/cpu:0"):

    def output_of_lambda(input_shape):
        shape = list(input_shape)
        return (shape[0], J)

    def weibull_cdf(parameters):
        m = parameters[:,0]
        s = tf.maximum( parameters[:,1], 0.001 )
        output_list = []
        for num in range( J ):
            Time   = tf.constant( num, dtype="float32")
            e_Time = tf.pow( Time, m )
            s_Time = tf.negative( tf.divide( e_Time, s) )
            x = tf.subtract( tf.constant(1, dtype="float32") , tf.exp( s_Time ) ) # F(t) = 1 - exp(-(t-g)^m/s) #ref http://www.mogami.com/notes/weibull.html
            output_list.append ( x )
        return tf.stack(output_list, axis=1)

    def generator_loss(y_true, y_pred, weights):  # y_true's shape=(batch_size, row, col, ch)
        #loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( y_pred, y_true ) ), weights ), axis=1, reverse=True)[:,0]
        log_p = tf.math.log( tf.add( y_pred,  tf.constant(1.0) ) )
        log_t = tf.math.log( tf.add( y_true,  tf.constant(1.0) ) )
        loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( log_p, log_t ) ), weights ), axis=1, reverse=True)[:,0]
        return loss

    def wrapped_generator_loss(func, *args, **kwargs):
        partial_generator_loss = partial(generator_loss, *args, **kwargs)
        update_wrapper(partial_generator_loss, generator_loss)
        return partial_generator_loss

    inputs = Input((n_features,), name='inputs')
    x1 = Dense(units=32, activation='relu', name='hidden_layer1',
                kernel_regularizer=regularizers.l1_l2(0.001))(inputs)
    x1 = Dropout(DROPOUT_RATIO)(x1)
    x2 = Dense(units=32, activation='relu', name='hidden_layer2',
                kernel_regularizer=regularizers.l1_l2(0.001))(x1)
    x2 = Dropout(DROPOUT_RATIO)(x2)
    x3 = Dense(units=32, activation='relu', name='hidden_layer3',
                kernel_regularizer=regularizers.l1_l2(0.001))(x2)
    x3 = Dropout(DROPOUT_RATIO)(x3)
    p1 = Dense(units=1, activation='softplus', name='param1_layer')(x3)
    p2 = Dense(units=1, activation='relu', name='param2_layer')(x3)
    parameters = Concatenate(name='params_layer')([p1, p2])
    y_pred = Lambda(weibull_cdf, output_shape=output_of_lambda)(parameters)

    mask_batch = Input((J,), name='mask_bartch')
    L = wrapped_generator_loss(generator_loss, weights = mask_batch)

    model = Model(inputs= [inputs, mask_batch], outputs = y_pred)
    model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 512)]        0           []                               
                                                                                                  
 hidden_layer1 (Dense)          (None, 32)           16416       ['inputs[0][0]']                 
                                                                                                  
 dropout_9 (Dropout)            (None, 32)           0           ['hidden_layer1[0][0]']          
                                                                                                  
 hidden_layer2 (Dense)          (None, 32)           1056        ['dropout_9[0][0]']              
                                                                                            

In [95]:
from tensorflow import keras
model = keras.models.load_model('/content/gdrive/MyDrive/dataset/Survival/Models/deepSurv2', compile=False)

## Training

In [89]:
pred_params = np.zeros([nn,2])
c_index_test = np.zeros([J,1])
outputfilename     = 'Training.csv'
weightfilename     = 'WeightBest.h5'
scaler = StandardScaler()
scaler.fit(x_train)
features_train = scaler.transform(x_train)
features_test = scaler.transform(x_test)
checkpointer = ModelCheckpoint(filepath=weightfilename, monitor='loss', verbose=1, save_best_only=True)

In [96]:
early_stopping = EarlyStopping(patience=10,verbose=1)
callbacks = []
callbacks.append(early_stopping)
adm = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adm, loss=L)
# model.run_eagerly = True
if mode != 'mat':
  model.fit([features_train, mask_train], y_train, batch_size=12, epochs = 100, callbacks=callbacks, verbose=0)
  model.save_weights('Model_Weights_.h5')

## Evaluation

In [ ]:

prob = model.predict([features_test, mask_test], batch_size=12)
c_index = np.zeros([J])
for num in range(J - 1) :
    c_index[num+1] = concordance_index(y_test[:,num+1],1/prob[:,num+1])
print( np.mean(c_index ))


## Save model and data

In [ ]:
scipy.io.savemat('/content/deep_surv_data/features_test.mat', {"data":x_test})
scipy.io.savemat('/content/deep_surv_data/features_train.mat', {"data":x_train})
scipy.io.savemat('/content/deep_surv_data/y_train.mat', {"data":y_train})
scipy.io.savemat('/content/deep_surv_data/y_test.mat', {"data":y_test})

scipy.io.savemat('/content/deep_surv_data/mask_test.mat', {"data":mask_train})
scipy.io.savemat('/content/deep_surv_data/mask_train.mat', {"data":mask_test})
scipy.io.savemat('/content/deep_surv_data/e_train.mat', {"data":e_train})
scipy.io.savemat('/content/deep_surv_data/e_test.mat', {"data":e_test})

In [ ]:
model.save('deepSurv')

# Model 4

## Load data

In [131]:
if mode == 'mat':
  x_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/features_gmv_train.mat')
  x_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/features_gmv_test.mat')
  d_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/d_gmv_train.mat')
  d_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/d_gmv_test.mat')
  e_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/event_gmv_train.mat')
  e_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/data_gmv/event_gmv_test.mat')
  x_train = np.array( x_train["data"], dtype = 'float32' )
  x_test = np.array( x_test["data"], dtype = 'float32' )
  d_train = np.array( d_train["data"], dtype = 'float32' ).flatten()
  d_test = np.array( d_test["data"], dtype = 'float32' ).flatten()
  e_train = np.array( e_train["data"], dtype = 'float32' ).flatten()
  e_test = np.array( e_test["data"], dtype = 'float32' ).flatten()

else:
  labels=list(zip(durations, event))
  x_train,x_test,y_train,y_test=train_test_split(covariates,labels,test_size=0.2)
  d_train=np.array([y[0] for y in y_train], dtype='float32')
  e_train=np.array([y[1] for y in y_train], dtype='float32')
  d_test=np.array([y[0] for y in y_test], dtype='float32')
  e_test=np.array([y[1] for y in y_test], dtype='float32')


In [132]:
print(x_train.shape, x_test.shape, d_train.shape, d_test.shape, e_train.shape, e_test.shape)

(201, 125440) (51, 125440) (201,) (51,) (201,) (51,)


## Network

In [135]:
labtrans = DeepHitSingle.label_transform(T)
labels = labtrans.fit_transform(d_train,e_train)

in_features = 125440
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1
batch_size=12
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
model2 = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

In [137]:
model2.load_net('/content/gdrive/MyDrive/dataset/Survival/Models/gmv_longitudnal.pt')

MLPVanilla(
  (net): Sequential(
    (0): DenseVanillaBlock(
      (linear): Linear(in_features=125440, out_features=32, bias=True)
      (activation): ReLU()
      (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): DenseVanillaBlock(
      (linear): Linear(in_features=32, out_features=32, bias=True)
      (activation): ReLU()
      (batch_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (2): Linear(in_features=32, out_features=6, bias=True)
  )
)

## Training

In [136]:
if mode != 'mat':
  model2.optimizer.set_lr(0.00009)
  epochs = 100
  callbacks = [tt.callbacks.EarlyStopping()]
  log = model2.fit(x_train, labels, batch_size, epochs, callbacks)

0:	[1s / 1s],		train_loss: 0.1929
1:	[1s / 3s],		train_loss: 0.1175
2:	[1s / 4s],		train_loss: 0.1034
3:	[1s / 5s],		train_loss: 0.0935
4:	[1s / 6s],		train_loss: 0.0902
5:	[1s / 7s],		train_loss: 0.0841
6:	[1s / 8s],		train_loss: 0.0759
7:	[1s / 10s],		train_loss: 0.0777
8:	[1s / 11s],		train_loss: 0.0718
9:	[1s / 12s],		train_loss: 0.0736


## Evaluation

In [138]:
surv = model2.interpolate(6).predict_surv_df(x_test)
ev = EvalSurv(surv, d_test, e_test, censor_surv='km')
ev.concordance_td('antolini')

0.7180851063829787

## Model Saving

In [ ]:
model2.save_net('gmv_longitudnal.pt')
scipy.io.savemat('/content/data/features_gmv_test.mat', {"data":x_test})
scipy.io.savemat('/content/data/features_gmv_train.mat', {"data":x_train})
scipy.io.savemat('/content/data/d_gmv_train.mat', {"data":d_train})
scipy.io.savemat('/content/data/event_gmv_train.mat', {"data":e_train})
scipy.io.savemat('/content/data/d_gmv_test.mat', {"data":d_test})
scipy.io.savemat('/content/data/event_gmv_test.mat', {"data":e_test})

# Model 5

## Load data

In [139]:
#replace with your path
if mode == 'mat':
  x_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/features_train.mat')
  x_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/features_test.mat')
  y_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/y_train.mat')
  y_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/y_test.mat')
  mask_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/mask_test.mat')
  mask_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/mask_train.mat')
  e_train=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/e_train.mat')
  e_test=scipy.io.loadmat('/content/gdrive/MyDrive/dataset/Survival/Data/genetic_data/e_test.mat')


  x_train = np.array( x_train["data"], dtype = 'float32' )
  x_test = np.array( x_test["data"], dtype = 'float32' )
  y_train = np.array( y_train["data"], dtype = 'float32' )
  y_test = np.array( y_test["data"], dtype = 'float32' )
  mask_train = np.array( mask_train["data"], dtype = 'float32' )
  mask_test = np.array( mask_test["data"], dtype = 'float32' )
  e_train = np.array( e_train["data"], dtype = 'float32' )
  e_test = np.array( e_test["data"], dtype = 'float32' )


else:
  x_train,x_test,y_train,y_test,mask_train,mask_test,e_train,e_test=train_test_split(x,E,mask,event,test_size=0.2)


## Network

In [140]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

with tf.device("/cpu:0"):

    def output_of_lambda(input_shape):
        shape = list(input_shape)
        return (shape[0], J)

    def weibull_cdf(parameters):
        m = parameters[:,0]
        s = tf.maximum( parameters[:,1], 0.001 )
        output_list = []
        for num in range( J ):
            Time   = tf.constant( num, dtype="float32")
            e_Time = tf.pow( Time, m )
            s_Time = tf.negative( tf.divide( e_Time, s) )
            x = tf.subtract( tf.constant(1, dtype="float32") , tf.exp( s_Time ) ) # F(t) = 1 - exp(-(t-g)^m/s) #ref http://www.mogami.com/notes/weibull.html
            output_list.append ( x )
        return tf.stack(output_list, axis=1)

    def generator_loss(y_true, y_pred, weights):  # y_true's shape=(batch_size, row, col, ch)
        #loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( y_pred, y_true ) ), weights ), axis=1, reverse=True)[:,0]
        log_p = tf.math.log( tf.add( y_pred,  tf.constant(1.0) ) )
        log_t = tf.math.log( tf.add( y_true,  tf.constant(1.0) ) )
        loss = tf.cumsum( tf.multiply( tf.square( tf.subtract( log_p, log_t ) ), weights ), axis=1, reverse=True)[:,0]
        return loss

    def wrapped_generator_loss(func, *args, **kwargs):
        partial_generator_loss = partial(generator_loss, *args, **kwargs)
        update_wrapper(partial_generator_loss, generator_loss)
        return partial_generator_loss

    inputs = Input((n_features,), name='inputs')
    x1 = Dense(units=32, activation='relu', name='hidden_layer1',
                kernel_regularizer=regularizers.l1_l2(0.001))(inputs)
    x1 = Dropout(DROPOUT_RATIO)(x1)
    x2 = Dense(units=32, activation='relu', name='hidden_layer2',
                kernel_regularizer=regularizers.l1_l2(0.001))(x1)
    x2 = Dropout(DROPOUT_RATIO)(x2)
    x3 = Dense(units=32, activation='relu', name='hidden_layer3',
                kernel_regularizer=regularizers.l1_l2(0.001))(x2)
    x3 = Dropout(DROPOUT_RATIO)(x3)
    p1 = Dense(units=1, activation='softplus', name='param1_layer')(x3)
    p2 = Dense(units=1, activation='relu', name='param2_layer')(x3)
    parameters = Concatenate(name='params_layer')([p1, p2])
    y_pred = Lambda(weibull_cdf, output_shape=output_of_lambda)(parameters)

    mask_batch = Input((J,), name='mask_bartch')
    L = wrapped_generator_loss(generator_loss, weights = mask_batch)

    model = Model(inputs= [inputs, mask_batch], outputs = y_pred)
    # model.summary()

In [143]:
from tensorflow import keras
model = keras.models.load_model('/content/gdrive/MyDrive/dataset/Survival/Models/genetic_model', compile=False)

## Training

In [144]:
pred_params = np.zeros([nn,2])
c_index_test = np.zeros([J,1])
outputfilename     = 'Training.csv'
weightfilename     = 'WeightBest.h5'
scaler = StandardScaler()
scaler.fit(x_train)
features_train = scaler.transform(x_train)
features_test = scaler.transform(x_test)
checkpointer = ModelCheckpoint(filepath=weightfilename, monitor='loss', verbose=1, save_best_only=True)

In [145]:
early_stopping = EarlyStopping(patience=10,verbose=1)
callbacks = []
callbacks.append(early_stopping)
adm = optimizers.Adam(lr=0.00003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adm, loss=L)
# model.run_eagerly = True
if mode != 'mat':
  model.fit([features_train, mask_train], y_train, batch_size=12, epochs = 100, callbacks=callbacks, verbose=0)
  model.save_weights('Model_Weights_' + str(num+1) + '.h5')

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Evaluation

In [146]:
prob = model.predict([features_test, mask_test], batch_size=12, verbose=1)

c_index = np.zeros([J])
for num in range(J - 1) :

    c_index[num+1] = concordance_index(y_test[:,num+1],1/prob[:,num+1])
print( np.mean(c_index ))

2/2 [==============================] - 0s 18ms/step
0.7090909090909092


## Model Save

In [ ]:

scipy.io.savemat('/content/genetic_data/features_test.mat', {"data":x_test})
scipy.io.savemat('/content/genetic_data/features_train.mat', {"data":x_train})
scipy.io.savemat('/content/genetic_data/y_train.mat', {"data":y_train})
scipy.io.savemat('/content/genetic_data/y_test.mat', {"data":y_test})

scipy.io.savemat('/content/genetic_data/mask_test.mat', {"data":mask_train})
scipy.io.savemat('/content/genetic_data/mask_train.mat', {"data":mask_test})
scipy.io.savemat('/content/genetic_data/e_train.mat', {"data":e_train})
scipy.io.savemat('/content/genetic_data/e_test.mat', {"data":e_test})

model.save('genetic_model')
